In [39]:
%matplotlib inline
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import numpy as np
import healpy as hp
import boost_histogram as bh
import os
import random
import time
import cupy as cp

In [15]:
def no_DN (frame):
    
    i = frame.index[0][0]
    new_frame = frame.loc[(frame['isDN'].values == False) & (frame['hitTime'].values < 300)]
    return new_frame.loc[i]

In [16]:
def first (frame):
    frame_sort = frame.sort_values(['hitTime'])
    frame_first = frame_sort.drop_duplicates(subset='pmtID', keep='first')
    frame_last = frame_first.sort_values('hitTime', ascending=False)
    return frame_last

In [17]:
def normal_ft(ev):
    
    mi = ev['hitTime'].min()
    
    a = ev['nHits'].values
    b = ev['pmtID'].values
    c = ev['hitTime'] - mi
    data = {'nHits': a, 'pmtID': b, 'hitTime': c}
        
    norm = pd.DataFrame(data)
    return norm

In [18]:
def mapev (ev, PMT_cor):
    
    x = PMT_cor['pmt_x']
    y = PMT_cor['pmt_y']
    z = PMT_cor['pmt_z']
    
    i = ev['pmtID'].values
    xs = x[ev['pmtID']]
    ys = y[ev['pmtID']]
    zs = z[ev['pmtID']] 
    t = ev['hitTime'].values
    data ={'pmtID': i, 'pmt_x': xs, 'pmt_y': ys, 'pmt_z':zs, 'hitTime': t}
    
    ev_map = pd.DataFrame(data)
    return ev_map

In [19]:
def mollweide (ev_map, j):
    # Set the coordinates for the input
    nside = 32
    npix = hp.nside2npix(nside)

    # Coordinates and the density field f
    hitTime = ev_map['hitTime']
    x = ev_map['pmt_x']
    y = ev_map['pmt_y']
    z = ev_map['pmt_z']
    
    # Go from HEALPix coordinates to indices
    indices = hp.vec2pix(nside, x, y, z)

    # Initate the map and fill it with the values
    hpxmap = np.full(npix, 0, dtype=np.float)
    hpxmap[[indices]]  = 300-hitTime
    cl = hp.anafast(hpxmap)

    # Inspect the map
    #project = hp.mollview(hpxmap, title=None, cbar=False, return_projected_map=True, hold=True) #,cmap='binary')
    #plt.savefig('/mnt/Storage/gvicentini/electron/ev'+str(j)+'.png')  
    #plt.clf()
    return cl #project

In [34]:
def save_hist (data, info, n, pre, b, position):
    
    dst_dir = '/home/centos/prova_E/E8_20PS/'
    
    hist = bh.Histogram(bh.axis.Regular(20, 0, 300))
    for i in info['evtID']:
        name = pre+str(i)+'_'+str(b)+'.npy'
        ev = data.pandas.df(entrystart=i, entrystop=i+1)
        try:
            ev_noDN = no_DN(ev)
        except:
            print(i, ': solo dark noise')
            continue
        ev_first = first(ev_noDN)
        ev_norm = normal_ft(ev_first)
        ev_time= ev_norm['hitTime'].values
        hist.fill(ev_time)
        array = hist.to_numpy()
        time = array[0]
        
        ev_map = mapev(ev_norm, position)
        cl = mollweide(ev_map, i)
        l = np.arange(len(cl))
        power = l * (l+1) * cl
        
        result = np.append(time, power)
        
        cp.save(dst_dir + name, result)
        hist.reset()

In [35]:
def call_all (file, a):
    
    data_file = uproot.open(file)
    INFO = data_file['true_info'].pandas.df()
    DATA = data_file["lpmt_hits"]
    POSITION = data_file['lpmt_pos'].pandas.df()
    
    if file.startswith('/home/centos/FlatComplete'):
        name = 'pos'
        
        INFO_E8 = INFO.loc[(INFO['E'] > 8) & (INFO['E'] < 9) ]
        
    elif file.startswith('/home/centos/ElectronDataset'):
        name = 'el'
        
        INFO_E8 = INFO.loc[(INFO['E'] > 9) & (INFO['E'] < 10) ]
        
    else:
        print('file non riconosciuto')
        
    save_hist(DATA, INFO_E8, 8, name, a, POSITION)

In [36]:
P0 = "/home/centos/FlatComplete/eplus_hits_dn_0.root"
P1 = "/home/centos/FlatComplete/eplus_hits_dn_1.root"
P2 = "/home/centos/FlatComplete/eplus_hits_dn_2.root"
P3 = "/home/centos/FlatComplete/eplus_hits_dn_3.root"
P4 = "/home/centos/FlatComplete/eplus_hits_dn_4.root"
P5 = "/home/centos/FlatComplete/eplus_hits_dn_5.root"
P6 = "/home/centos/FlatComplete/eplus_hits_dn_6.root"
P7 = "/home/centos/FlatComplete/eplus_hits_dn_7.root"
P8 = "/home/centos/FlatComplete/eplus_hits_dn_8.root"
P9 = "/home/centos/FlatComplete/eplus_hits_dn_9.root"

In [37]:
E0 = "/home/centos/ElectronDataset/Hits/eminus_hits_dn_0.root"
E1 = "/home/centos/ElectronDataset/Hits/eminus_hits_dn_1.root"
E2 = "/home/centos/ElectronDataset/Hits/eminus_hits_dn_2.root"
E3 = "/home/centos/ElectronDataset/Hits/eminus_hits_dn_3.root"
E4 = "/home/centos/ElectronDataset/Hits/eminus_hits_dn_4.root"
E5 = "/home/centos/ElectronDataset/Hits/eminus_hits_dn_5.root"
E6 = "/home/centos/ElectronDataset/Hits/eminus_hits_dn_6.root"
E7 = "/home/centos/ElectronDataset/Hits/eminus_hits_dn_7.root"
E8 = "/home/centos/ElectronDataset/Hits/eminus_hits_dn_8.root"
E9 = "/home/centos/ElectronDataset/Hits/eminus_hits_dn_9.root"

In [40]:
time.strftime("%H") +":"+ time.strftime("%M")

'09:57'

In [41]:
call_all(P0, 0)
call_all(E0, 0)
time.strftime("%H") +":"+ time.strftime("%M")

/home/centos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


'10:31'

In [42]:
call_all(P1, 1)
call_all(E1, 1)
time.strftime("%H") +":"+ time.strftime("%M")

/home/centos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


'11:05'

In [43]:
call_all(P2, 2)
call_all(E2, 2)
time.strftime("%H") +":"+ time.strftime("%M")

/home/centos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


'11:38'

In [44]:
call_all(P3, 3)
call_all(E3, 3)
time.strftime("%H") +":"+ time.strftime("%M")

/home/centos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


'12:12'

In [45]:
call_all(P4, 4)
call_all(E4, 4)
time.strftime("%H") +":"+ time.strftime("%M")

/home/centos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


'12:46'

In [46]:
call_all(P5, 5)
call_all(E5, 5)
time.strftime("%H") +":"+ time.strftime("%M")

/home/centos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


'13:20'

In [47]:
call_all(P6, 6)
call_all(E6, 6)
time.strftime("%H") +":"+ time.strftime("%M")

/home/centos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


'13:54'

In [48]:
call_all(P7, 7)
call_all(E7, 7)
time.strftime("%H") +":"+ time.strftime("%M")

/home/centos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


'14:29'

In [49]:
call_all(P8, 8)
call_all(E8, 8)
time.strftime("%H") +":"+ time.strftime("%M")

/home/centos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


'15:02'

In [50]:
call_all(P9, 9)
call_all(E9, 9)
time.strftime("%H") +":"+ time.strftime("%M")

/home/centos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


'15:36'